# Reforumlated Markowitz Model - Optimization Codes through Quantum Computing

Import the necessary libraries

In [2]:
import numpy as np
import pandas as pd
from dimod import ConstrainedQuadraticModel, Integer, Binary
import time

Importing the Data

In [3]:
df = pd.read_excel("Daily_Returns.xlsx")

Changing the Date to the index

In [4]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Calculating the expected returns and the variance covariance matrix

In [5]:
# Calculate expected returns for each asset
expected_returns = np.mean(df,axis=0)*252

# Calculate the variance-covariance matrix
cov_matrix = df.cov()


#Converting the expected returns to a numpy array
expected_returns = expected_returns.values

#The number of assets
n_assets = len(expected_returns)

Setting the values of the constraints

In [6]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 0.1

# Maximum units of each asset that can be purchased (D)
D = 5

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 15

# The number of shares the investor would like to purchase
S = 20

Creating the Decision Variables

In [7]:
y = [Binary(f"y_{i}") for i in range(n_assets)] #To represent y

In [8]:
X = [Integer(f"x_{i}") for i in range(n_assets)] #To represent x

Finding the weights

In [9]:
W = [x/S for x in X] 

Create the CQM Object

In [10]:
cqm = ConstrainedQuadraticModel()

Creating the Objective Function

In [11]:
H_obj = 0
for i in range(n_assets):
    H_obj -= expected_returns[i] * W[i]

Creating the risk constraint

In [12]:
# Convert the Variance Covariance matrix to a NumPy array
cov_matrix_np = cov_matrix.to_numpy()

# Convert X to a NumPy array
W_numeric = np.array(W)  

# Calculate the total variance
total_variance = np.dot(W_numeric.T, np.dot(cov_matrix_np, W_numeric))

Setting the Objective Function

In [13]:
cqm.set_objective(H_obj)

Setting the Risk Constraint

In [14]:
cqm.add_constraint_from_model(total_variance, '<=', R**2, "Variance", weight=1)

'Variance'

Adding the Linking Constraints

In [15]:
# Diversification Constraints
for i in range(n_assets):
    cqm.add_constraint_from_model(X[i] - D*y[i], '<=', 0, label=f"Linking_{i}", weight=1)

Adding the cardinality constraint

In [16]:
cqm.add_constraint_from_model(sum(y), '<=', K, label="Cardinality", weight=1)

'Cardinality'

Adding the constraint on the number of shares

In [17]:
cqm.add_constraint_from_model(sum(X), '==', S, label="Total Shares", weight=1)

'Total Shares'

Solving the optimization problem

In [18]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token="DEV-df309da97162b976954c423a5c0df694bc7b7bdf")  

In [19]:
#Solve the problem
sampleset = sampler.sample_cqm(cqm)

In [20]:
print(sampleset.first) 

Sample(sample={'x_0': 0.0, 'x_1': 5.0, 'x_10': 0.0, 'x_100': 0.0, 'x_101': 0.0, 'x_102': 0.0, 'x_103': 0.0, 'x_104': 0.0, 'x_105': 0.0, 'x_106': 0.0, 'x_107': 0.0, 'x_108': 0.0, 'x_109': 0.0, 'x_11': 0.0, 'x_110': 0.0, 'x_111': 0.0, 'x_112': 0.0, 'x_113': 0.0, 'x_114': 0.0, 'x_115': 0.0, 'x_116': 0.0, 'x_117': 0.0, 'x_118': 0.0, 'x_119': 0.0, 'x_12': 0.0, 'x_120': 0.0, 'x_121': 0.0, 'x_122': 0.0, 'x_123': 0.0, 'x_124': 0.0, 'x_125': 0.0, 'x_126': 0.0, 'x_127': 0.0, 'x_128': 0.0, 'x_129': 0.0, 'x_13': 0.0, 'x_130': 0.0, 'x_131': 0.0, 'x_132': 0.0, 'x_133': 0.0, 'x_134': 0.0, 'x_135': 0.0, 'x_136': 0.0, 'x_137': 0.0, 'x_138': 0.0, 'x_139': 0.0, 'x_14': 0.0, 'x_140': 0.0, 'x_141': 0.0, 'x_142': 0.0, 'x_143': 0.0, 'x_144': 0.0, 'x_15': 0.0, 'x_16': 0.0, 'x_17': 0.0, 'x_18': 0.0, 'x_19': 0.0, 'x_2': 0.0, 'x_20': 0.0, 'x_21': 0.0, 'x_22': 0.0, 'x_23': 0.0, 'x_24': 0.0, 'x_25': 0.0, 'x_26': 0.0, 'x_27': 0.0, 'x_28': 0.0, 'x_29': 0.0, 'x_3': 0.0, 'x_30': 0.0, 'x_31': 0.0, 'x_32': 5.0, 'x_33': 

Checking for the violation of constraints

In [21]:
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)

Recreating the original decision variables

In [22]:
sample = sampleset.first[0]

# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])

decision_variables


[0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

Calculating the Return

In [23]:
# Find the weights
result = [x / S for x in decision_variables]


# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_returns, result))

print(total)

0.20186450006434886


Now let us test to see if all the constraints are satisfied

In [24]:
# Extract y values from sample
y_optimized = np.array([sample[f'y_{i}'] for i in range(n_assets)])

# Convert decision_variables to a numpy array for easy manipulation
x_optimized = np.array(decision_variables)
w_optimized = np.array(result)

# Now, checking the constraints with these values
# Constraint 1: Risk constraint
risk_value = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))
print(f"Risk Constraint Satisfied: {risk_value <= R}")
print(f"Risk Value: {risk_value} <= {R}")

# Constraint 2: Linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}")


# Constraint 5: Sum of units
unit_sum = sum(x_optimized)
print(f"Sum of units Constraint Satisfied: {unit_sum <= S}")
print(f"Number of units Included: {unit_sum} <= {S}")


Risk Constraint Satisfied: True
Risk Value: 0.014684182951557738 <= 0.1
Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 4 <= 15
Non-negativity Constraints Satisfied: True
Sum of units Constraint Satisfied: True
Number of units Included: 20.0 <= 20


Checking the assets to invest in

In [25]:
values_list = x_optimized.tolist()

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Asset': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Asset'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)

  Asset  Value
0  AAPL    5.0
1   ARE    5.0
2  BIIB    5.0
3   CHD    5.0


Calculating the portfolio with the highest returns

In [32]:
#Sum of shares
S = 1000

# Create the CQM Object
cqm = ConstrainedQuadraticModel()

#Adding the objective function
cqm.set_objective(H_obj)


#Adding the sum of shares constraint
cqm.add_constraint_from_model(sum(X), '==', S, label="Total Shares", weight=1)

#Solve the problem
sampleset = sampler.sample_cqm(cqm)



#Checking for violations
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)



#Recreating the original variables
sample = sampleset.first[0]

# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])




#Calculating the total return

# Find the weights
weights = [x / S for x in decision_variables]


# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_returns, weights))

# Convert decision_variables to a numpy array for easy manipulation
x_optimized = np.array(decision_variables)

# Convert weights to a numpy array for easy manipulation
w_optimized = np.array(weights)

# Risk Value
risk_value_maximum = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))


print(x_optimized)
print("Risk: ", risk_value_maximum)
print("Return: ", total)

[ 1. 10.  0. 11.  4.  9. 21. 14. 39.  0.  3.  7.  3. 19. 18.  0. 13. 14.
  3.  4.  1.  6. 15. 16. 11.  0. 16. 25.  6.  5.  4. 24.  6.  0.  0. 38.
  1.  9.  2. 12.  2.  2.  7.  6.  0.  6. 11.  3.  0. 19.  6.  1. 20.  3.
  3.  2.  7.  6.  5.  8.  6.  2.  2. 15.  4.  7.  8.  9.  4.  7. 20.  2.
  2.  1.  1.  8.  8.  0.  1.  5.  6.  2.  1.  0.  1.  0.  8.  2. 18. 17.
  7. 19.  0.  0.  0.  5.  0.  0.  7.  4.  0.  0.  2.  1. 15.  7.  4.  1.
 48.  1.  2.  4.  4.  0.  0.  5.  8. 10.  3.  0.  1.  5.  2.  4.  3. 10.
  0. 13.  2.  6.  7.  1.  6. 16.  1. 19.  2. 24. 14. 13.  0.  7.  4.  5.
  4.]
Risk:  0.011729974780896943
Return:  0.16444141306992976


Calculating the portolio with the lowest risk

In [30]:
#Sum of shares
S = 1000



# Create the CQM Object
cqm = ConstrainedQuadraticModel()

#Adding the objective function
cqm.set_objective(total_variance)


#Adding the sum of shares constraint
cqm.add_constraint_from_model(sum(X), '==', S, label="Total Shares", weight=1)

#Solve the problem
sampleset = sampler.sample_cqm(cqm)



#Checking for violations
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)



#Recreating the original variables
sample = sampleset.first[0]

# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])




#Calculating the total return

# Find the weights
weights = [x / S for x in decision_variables]


# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_returns, weights))

# Convert decision_variables to a numpy array for easy manipulation
x_optimized_MIN_RISK = np.array(decision_variables)

# Convert weights to a numpy array for easy manipulation
w_optimized_MIN_RISK = np.array(weights)

# Risk Value
risk_value_minimum = np.sqrt(np.dot(np.dot(w_optimized_MIN_RISK.T, cov_matrix), w_optimized_MIN_RISK))


print(x_optimized_MIN_RISK)
print("Risk: ", risk_value_minimum)
print("Return: ", total)

[ 9.  0.  0.  8.  1.  8.  7. 13. 17.  6.  0. 10.  0.  6.  0.  0.  0.  1.
  4. 12.  2.  0.  3.  0.  2.  0.  0.  1.  2.  5.  0. 13.  2.  8.  5.  1.
 11.  5.  4.  4.  3.  3.  5.  7.  4. 12.  9.  5.  1.  1. 22.  4. 48.  2.
  2.  5.  9.  0. 18.  0.  3. 14. 16.  1.  4.  0.  1. 18. 44.  5. 13.  6.
 12.  0.  2.  2. 17. 29.  0.  4.  1.  6.  1. 17.  8.  4.  1. 15. 17.  0.
  4.  6.  6.  3.  1.  8. 25. 20.  7. 34.  8.  2.  4. 12.  9.  2.  1. 14.
 10.  0.  2.  5. 15. 16.  1.  5.  7. 11.  3.  3. 20.  3.  2. 11.  2. 11.
 12.  3.  8.  1.  0. 18.  0.  0.  1.  4.  4.  0. 12.  5.  4. 13. 20.  3.
  8.]
Risk:  0.010422696063259025
Return:  0.14650706878137298


Calculating the efficient frontier

In [38]:
# Create the CQM Object
cqm = ConstrainedQuadraticModel()


tolerance = 0.001  # define tolerance level for risk


In [39]:
# Continue to set your objective for maximizing returns
cqm.set_objective(H_obj)

# Constraint for total shares remains unchanged
cqm.add_constraint_from_model(sum(X), '==', S, label="Total Shares")

# Add risk constraints
cqm.add_constraint_from_model(total_variance, '>=' ,(risk_value_minimum - tolerance)**2, label="Min Risk",weight=1)
cqm.add_constraint_from_model(total_variance, '<=' ,(risk_value_maximum + tolerance)**2, label="Max Risk",weight=1)



#Solve the problem
sampleset = sampler.sample_cqm(cqm)



#Checking for violations
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)



#Recreating the original variables
sample = sampleset.first[0]

# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])




'Max Risk'

Looping it all

In [83]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
Variance_Values = [i / 100 for i in range(10, 21)]

#Sum of shares
S = 1000

In [84]:
"""

We will now loop over multiple risk values to calculate the return and get an efficient frontier and generate
each of the portfolios for these risk values. Note that all the constraints except the number of shares the investor should invest
in would be relaxed in this solution for the purpose of comparing with the vanilla markowitz model.

"""

results_list = []  # Initialize the list

for R in Variance_Values:

    
    # Check the variance value
    print(R)

    # Create the CQM Object
    cqm = ConstrainedQuadraticModel()

    #Adding the objective function
    cqm.set_objective(H_obj)

    #Adding the risk constraint 
    cqm.add_constraint_from_model(total_variance, '<=', R**2, "Variance", weight=1)

    
    #Adding the sum of shares constraint
    cqm.add_constraint_from_model(sum(X), '==', S, label="Total Shares", weight=1)


    # Time the sampler
    start_time = time.time()
    sampleset = sampler.sample_cqm(cqm)
    time_taken = time.time() - start_time


    #Checking for violations
    for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
        print(label, violation)


    #Recreating the original variables
    sample = sampleset.first[0]

    # Number of decision variables
    num_decision_variables = n_assets

    # Reconstruct the original decision variables
    decision_variables = []
    for i in range(num_decision_variables):
        decision_variables.append(sample[f'x_{i}'])


    #Calculating the total return

    # Find the weights
    weights = [x / S for x in decision_variables]


    # Multiply corresponding elements and sum them up
    total = sum(x * y for x, y in zip(expected_returns, weights))

    # Convert decision_variables to a numpy array for easy manipulation
    x_optimized = np.array(decision_variables)

    # Convert weights to a numpy array for easy manipulation
    w_optimized = np.array(weights)

    # Constraint 1: Risk constraint
    risk_value = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))

    # Constraint 2: Non-negativity constraint
    non_negativity_satisfied = all(x_optimized >= 0)

    # Constraint 3: Sum of shares constraint
    sum_of_shares_satisfied = sum(x_optimized) == S

    #Creating the asset dataframe
    values_list = x_optimized.tolist()

    # Creating a new DataFrame with column names and values where values are > 0
    filtered_data = {'Asset': [], 'Value': []}
    for column_name, value in zip(df.columns, values_list):
        if value > 0:
            filtered_data['Asset'].append(column_name)
            filtered_data['Value'].append(value)

    # Create a dictionary for the current iteration
    iteration_results = {
        'time_taken': time_taken,
        'risk_value': risk_value,
        'R': R,
        'total_return': total,
        'risk_constraint_satisfied': risk_value <= R,
        'non_negativity_satisfied': non_negativity_satisfied,
        'Total_Shares_satisfied': sum_of_shares_satisfied,
        'assets_included_df': filtered_data
    }

    # Step 4: Append this dictionary to the list
    results_list.append(iteration_results)

0.1
Variance 0.27000436532009675
0.11
Variance 0.2670000078462849
0.12
Variance 0.1662061972760804
0.13
Variance 0.21982535703582753
0.14
Variance 0.2878997976072794
0.15
Variance 0.24980844305824842
0.16
Variance 0.24672097194417303
0.17
Variance 0.23945784207905388
0.18
Variance 0.25984832555729326
0.19
Variance 0.26272333196141107
0.2
Variance 0.2428107131981698


Creating a dataframe of results

In [85]:
# Creating a DataFrame from results_list for specific metrics
metrics_columns = [
    'time_taken',
    'risk_value',
    'R',
    'total_return',
    'risk_constraint_satisfied',
    'non_negativity_satisfied',
    'Total_Shares_satisfied'
]

# Use a list comprehension to pick out these specific keys from each dictionary in the results list
final_results_data = [{col: result[col] for col in metrics_columns} for result in results_list]

# Convert this list of dictionaries to a DataFrame
final_results_df = pd.DataFrame(final_results_data)

# Save the DataFrame as an Excel file
final_results_df.to_excel('Final_Results.xlsx', index=False)


Creating the excels of Asset Allocations

In [86]:
for i, result in enumerate(results_list, start=1):
    # Convert filtered_data (which replaced new_df) to a DataFrame
    assets_df = pd.DataFrame(result['assets_included_df'])
    
    # Save the DataFrame as an Excel file
    file_name = f'Assets{i}.xlsx'
    assets_df.to_excel(file_name, index=False)